# ML Example

### Identify the problem
What is the purpose of the project, in the case of house price forecasting, then the purpose is to forecast the median property price

### Solutions
- What is the business purpose, after all, building models is not the ultimate purpose
- To know how effective the current solution is, for example, will give an error rate of the current solution is alpha
- One can further study the problem to clarify whether the problem is supervised/unsupervised, or a reinforcement model? Is it classification/regression, or other such as clustering. To use batch learning or online learning?
- Example: We have house price values, so it is a supervised problem; we eventually want to predict the median house price, so it is a regression problem, and it is a multivariate predictive regression because there are many influencing parameters; in addition, there is no continuous inflow of data, and there is no special need to make quick adaptation to data changes. The amount of data is not large enough to be put into memory, so batch learning is fine. If the data volume is large, you can either split the batch learning across multiple servers (using MapReduce techniques, as you will see later), or use online learning].

### Select performance indicators
Here we need to choose an evaluation metric, typical for regression problems is the root mean squared error RMSE, which characterizes the standard deviation of the system prediction error.
Alternatively, the difference squared absolute error can be used.

### Check assumptions
- Check the data, for example, check the data type, check the missing data, check the outliers, check the distribution of the data, check the correlation between the data, check the data quality, etc.

### Create workspace

such as python jupyter and corresponding library files (such as numpy, pandas, scipy, and sklearn, etc.) and frameworks (tf, etc.)

### Get data
- Download the data
- Load the data
- Explore the data
- Visualize the data

In [ ]:
import pandas as pd
data=pd.read_csv('housing.csv')
data.head()
data.info()
data.describe()


import matplotlib.pyplot as plt
data.hist(bins=50, figsize=(20,15))
plt.show()



### Creating a test set

Before viewing the data, it is best to create a test set so that the selection of the test set is not affected by the mindset after viewing the data.
One way is that you can choose the test set randomly, for example, choose 20% of the data randomly as the test set, but then when the data set is updated, the test set will change and we can use random numbers to handle it.

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)


In addition, there is a risk of losing the distribution of key features by random sampling. For example, there is a feature A that contributes a lot to the final label (the correlation between them is strong).
Then we should also ensure in the test set that the distribution of A follows the distribution trend of the original dataset. This can be done using stratified sampling

In [ ]:
data["A_new"] = np.ceil(data["A"] / 1.5)
data["A_new"].where(data["A_new"] < 5, 5.0, inplace=True)
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(data, data["A_new"]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

#The results of stratified sampling can be checked with the following code.
data["A_new"].value_counts() / len(data)
strat_test_set["A_new"].value_counts() / len(strat_test_set)

# Note that the generated A_new tag needs to be removed at the end, with the drop command, code.
for set in (strat_train_set, strat_test_set):
    set.drop(["A_new"], axis=1, inplace=True)

### Data Visualization

Frequently, we need to first take a look at the data and can determine how the features relate to the labels and which features are more useful or influential. The matplotlib library can be used here

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")


### Finding correlations

You can use the corr() method to calculate the standard correlation coefficient between each pair of attributes

In [ ]:
corr_m=housing.corr()
print(corr_m['median_house_value'].sort_values(ascending=True))


Note that corr() can only characterize linear relationships and ignore non-linear relationships, so there are limitations in its referential properties. There is another way to detect correlation coefficients between attributes when pandas' scatter_matrix function

In [ ]:
from pandas.tools.plotting import scatter_matrix
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))


### Attribute combination test

Sometimes just using the original feature data does not work well, it is possible to consider combining some features to generate new features, such as the number of people / households, to get a feature such as the number of people per household

In [ ]:
housing['population_per_household']=housing['population']/housing['households']


### Data Cleaning

Problems such as missing values can exist in the original data, so the data needs to be cleaned. This is a very critical step.
There are three ways to deal with missing values
1. directly delete the row where the missing value is located; 
2. if a feature has too many missing values, then directly delete the feature; 
3. assign a value to the missing position (with 0, median or average, etc.)

In [ ]:
housing.dropna(housing['total_bedrooms'])
housing.drop('total_bedrooms',axis=1)
housing['total_bedrooms'].fillna(median)


In [ ]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median")
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)


In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
print(housing_cat_1hot)


In [ ]:
cat_encoder = CategoricalEncoder()
housing_cat_reshaped = housing_cat.values.reshape(-1, 1)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat_reshaped)
print(housing_cat_1hot)


In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")),
    ])

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])


In [ ]:
housing_prepared=full_pipeline.fit_transform(housing)


In [ ]:

from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
from from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
def display_scores(scores):
...     print("Scores:", scores)
...     print("Mean:", scores.mean())
...     print("Standard deviation:", scores.std())
display_scores(tree_rmse_scores)


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)


print(grid_search.best_params_)
print(grid_search.best_estimator_)


In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
print(feature_importances)

extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = list(encoder.classes_)
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances,attributes), reverse=True)
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) 
print(final_rmse)
